In [0]:
from pyspark.sql import SparkSession, functions as f

claims_df = spark.read.csv("/mnt/landing/claims/*.csv", header=True)

claims_df = claims_df.withColumn(
    "datasource",
    f.when(f.col("_metadata.file_path").contains("hospital1"), "hosa")
     .when(f.col("_metadata.file_path").contains("hospital2"), "hosb")
     .otherwise(None)
)

display(claims_df)

# Parquet file creation
claims_df.write.format("parquet").mode("overwrite").save("/mnt/bronze/claims/")



claims_df.createOrReplaceTempView("claims_df")

In [0]:
from pyspark.sql.functions import col

dfRenamed = claims_df.toDF(*[col_name.replace(" ", "_") for col_name in claims_df.columns])


In [0]:
%sql
create schema if not exists bronze

In [0]:
dfRenamed.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("databricksdev.bronze.claims_data")

In [0]:
%sql
select * from bronze.claims_data